In [6]:
import requests
import pandas as pd
import json

In [7]:
# get keys from file
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [9]:

keys = get_keys("/API/api_football.json")
api_key = keys['api_key']

In [10]:
# def for url
headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': api_key
    }

def api_call(url):
    response = requests.request("GET", url, headers=headers)
    # Return a dictionary of the response text
    return json.loads(response.text)

In [ ]:
# create base url
base_url = 'https://api-football-v1.p.rapidapi.com/v2/'
leagues_url = base_url + 'leagues/'

In [ ]:
# get data response of available leagues
available_leagues = api_call(leagues_url)

In [ ]:
# transform to pandas dataframe
leagues_df = pd.DataFrame.from_dict(available_leagues['api']['leagues'])

In [ ]:
# the id of current England Premier League
premier_league_id = leagues_df[(leagues_df.country == 'England') & (leagues_df.season == 2020) & 
                               (leagues_df.name == 'Premier League')].league_id
premier_league_id.values[0]

In [ ]:
league_fixtures = base_url + 'fixtures/league/' + str(league_en_2020_id.values[0])
league_fixtures_en_2020 = api_call(league_fixtures)

In [ ]:
# save data leagues of matches for future (01_02_collect_data)
with open('league_fixtures_2020.txt', 'w') as f:
    json.dump(league_fixtures_en_2020, f, ensure_ascii=False)

In [ ]:
# Define an object orientated approach for a match
class Match():
    def __init__(self, match_json):
        self.match_json = match_json
        self.home_team = match_json['homeTeam']['team_name']
        self.away_team = match_json['awayTeam']['team_name']
        self.fixture_id = match_json['fixture_id']
        
    def fixture(self):
        return(self.home_team + ' vs. ' + self.away_team)
    
    def final_score(self):
        return(self.match_json['score']['fulltime'])

In [ ]:
class Events:
    def __init__(self, match_events_json):
        self.n_events = match_events_json['results']
        self.events = match_events_json['events']
        events_df = pd.DataFrame.from_dict(match_events_json['events'])
        self.events_df = events_df
        
    def goals(self):
        goals = self.events_df[self.events_df['type'] == 'Goal'][['type', 'detail', 'player', 'teamName', 'elapsed']].set_index('elapsed')
        goals.index.rename('mins', inplace=True)
        return(goals)

In [ ]:
# collect data 


In [ ]:
base_url = 'https://api-football-v1.p.rapidapi.com/v2/'
events_url = base_url + 'events/'

import time

num1=0
num2=380
league_fixtures = league_fixtures_2020

for number in range(num1,num2):
    # set time sleep 3 second
    time.sleep(3)
    
    number_game = league_fixtures['api']['fixtures'][number]
    number_game_obj = Match(number_game)
    number_game_url = events_url + str(number_game_obj.fixture_id)
    events = api_call(number_game_url)['api']
    number_df = pd.DataFrame(data=Events(events).goals(), columns=None)
    number_df['id_match'] = number_game_obj.fixture_id
    df_ex = number_df
    if number>num1:
        df = pd.concat([df, df_ex])
    else:
        df = df_ex

In [ ]:
df_en_2020_mins = df.reset_index()